<a href="https://colab.research.google.com/github/Silverz21/Ciencia-de-Datos/blob/main/Tarea_2_Muestreo_Sa%C3%BAl_Santos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [446]:
import pandas as pd
import numpy as np
import scipy.stats as st

Fijar un conjunto de datos en donde se mida una variable de interés sobre una 
población. Describir dicho conjunto de datos, detectar la presencia de posibles
valores atípicos y de acuerdo a la variable que se esta midiendo, definir un
atributo para estimar una proporción o total poblacional.

En este ejemplo nuestros datos son los adeudos que ciertas compañias tienen con la nuestra (esto en dólares). El atributo a estimar para la muestra serán las compañias que tengan un adeudo menor a los $\$$2000 dólares y a su vez el de la
proporción serán aquellas que tengan adeudo menor a $600 dólares.

Como primer paso extraemos nuestro conjunto de datos, los cuáles están dentro
de un archivo de excel. Después creamos un DataFrame (o una matriz con los 
datos de nuestro archivo), el cual es con el que vamos a trabajar.

In [447]:
path = 'datos.xlsx'
df = pd.read_excel(path)
df=df.set_index("ID")
N = df.count()[0]
df

,Proveedor,Saldo pendiente (en dólares)
ID,,
P0001,So Factive,63.0
P0002,Kontroller,4250.0
P0003,Finance Api,2412.0
P0004,Biomotivate,3348.0
P0005,Deltavita,570.0
...,...,...
P0995,Actenet,7876.0
P0996,Datawrite,4703.0
P0997,Echo Digital,2786.0


Nombramos como muestra a los elementos de interés de la base de datos, los cuales son los saldos menores a 2000 dólares.

In [448]:
muestra = df.groupby(level=["ID"]).filter(lambda x : x["Saldo pendiente (en dólares)"]<2000)
muestra = muestra.reset_index()
muestra = muestra["Saldo pendiente (en dólares)"].squeeze()
muestra

0        63.00
1       570.00
2       261.00
3       310.00
4        71.14
        ...   
218    1099.00
219    1678.00
220     980.00
221     329.00
222    1201.00
Name: Saldo pendiente (en dólares), Length: 223, dtype: float64

Pedimos los valores de la encuesta piloto, el margén de error y la probabilidad permisible.

In [468]:
valido = False

def validar(v):
	entero = int(v)
	return entero >= 10 and entero <= 20

print('Introduzca el tamaño de la encuesta de la encuesta piloto (de 10 a 20): ', end='')

while not valido:
	v = input()
	valido = validar(v)
	if not valido:
		print('Lo siento, el valor no esta dentro del intervalo, vuelva a intentarlo: ', end='')

print(f'El tamaño de la encuesta piloto es {v}.')
v=pd.to_numeric(v)

Introduzca el tamaño de la encuesta de la encuesta piloto (de 10 a 20): 20
El tamaño de la encuesta piloto es 20.


In [469]:
valido1 = False

def validar1(me):
	entero1 = float(me)
	return entero1 >= 0 and entero1 <= 1

print('Introduzca el margen de error (entre 0 y 1): ', end='')

while not valido1:
	me = input()
	valido1 = validar1(me)
	if not valido1:
		print('Lo siento, el valor no esta dentro del intervalo, vuelva a intentarlo: ', end='')

print(f'El margen de error es {me}.')

me = pd.to_numeric(me)

Introduzca el margen de error (entre 0 y 1): .05
El margen de error es .05.


In [487]:
valido2 = False

def validar2(alfa):
	entero2 = float(alfa)
	return entero2 >= 90 and entero2 <= 99

print('Introduzca el porcentaje error permisible (entre 90 y 99): ', end='')

while not valido2:
	alfa = input()
	valido2 = validar2(alfa)
	if not valido2:
		print('Lo siento, el valor no esta dentro del intervalo, vuelva a intentarlo: ', end='')

print(f'El porcentaje error permisible es {alfa}%.')
alfa = pd.to_numeric(alfa)
alfa = 1-(alfa/100)

Introduzca el porcentaje error permisible (entre 90 y 99): 95
El porcentaje error permisible es 95%.


En base al tamaño de la encuesta piloto obtenemos (aleatoriamente) datos de nuestra muestra.

In [488]:
ep = pd.Series(np.random.choice(muestra, v))

Y ahora obtenemos la proporción (saldos menores a 600 dólares) a partir de nuestros elementos obtenidos por la encuesta piloto.

In [489]:
ep = pd.DataFrame(ep)
ep.rename(columns={0:'Saldo pendiente'},
                inplace=True)

In [490]:
ep1 = ep.groupby(level=[0]).filter(lambda x : x["Saldo pendiente"]<600)

In [491]:
prop = ep1.count()/ep.count()

El valor z con el porcentaje de error permisible también lo calculamos.

In [492]:
import scipy.stats
valorz = scipy.stats.norm.ppf(alfa/2)

Y así obtenemos el tamaño de la muestra con la proporción estimada (redondeamos el número).

In [493]:
tamañomuestra = (valorz**2 * N * prop * (1-prop)) / (((N-1) * me**2) + (valorz**2 * prop * (1-prop)))
tamañomuestra = tamañomuestra.squeeze()
tamañomuestra = round(tamañomuestra)

Y también obtenemos el tamaño de la muestra con la proporción $\frac{1}{2}$.


In [494]:
tammues = (valorz**2 * N * .5 * (.5)) / (((N-1) * me**2) + (valorz**2 * .5 * (1-.5)))
tammues = round(tammues)

Ahora calculamos la proporción muestral con el primer tamaño calculado.

In [495]:
epmues1 = pd.Series(np.random.choice(muestra, tamañomuestra))
epmues1 = pd.DataFrame(epmues1)
a = epmues1.count()

In [496]:
epmues1 = epmues1.groupby(level=[0]).filter(lambda x : x[0]<600)
b = epmues1.count()
epmues1 = epmues1.squeeze()

In [497]:
propmues1 = b/a
propmues1 = propmues1.squeeze()

También se calcula el error muestral y el intervalo de confianza.

In [498]:
errormues1 = np.sqrt( ( (N-a)/(N-1) ) * ( (propmues1*(1 - propmues1)/a) ) )
errormues1 = errormues1.squeeze()


In [499]:
interconfmues1 =  st.norm.interval (alpha = alfa , loc = propmues1)

Y hacemos lo mismo con la proporción de $\frac{1}{2}$.

In [500]:
epmues2 = pd.Series(np.random.choice(muestra, tammues))
epmues2 = pd.DataFrame(epmues2)
a2 = epmues2.count()
epmues2
epmues2 = epmues2.groupby(level=[0]).filter(lambda x : x[0]<600)
b2 = epmues2.count()
epmues2 = epmues2.squeeze()
epmues2
propmues2 = b2/a2
propmues2 = propmues2.squeeze()
propmues2
errormues2 = np.sqrt( ( (N-a)/(N-1) ) * ( (propmues2*(1 - propmues2)/a) ) )
errormues2 = errormues2.squeeze()
errormues2
interconfmues2 =  st.norm.interval (alpha = alfa , loc = propmues2)

Y ahora imprimimos los resultados.

In [501]:
print(f'El tamaño de la encuesta piloto es: {v}.')
print(f'El margen de error es: {me}.')
print(f'El porcentaje error permisible es: {100-round((alfa*100))}%.')
print(f'La proporción de la encuesta piloto es: {prop.squeeze()}.')

El tamaño de la encuesta piloto es: 20.
El margen de error es: 0.05.
El porcentaje error permisible es: 95%.
La proporción de la encuesta piloto es: 0.2.


In [502]:
print(f"El tamaño de la muestra 1 es: {tamañomuestra}")
print(f"La proporción de la muestra 1 es: {propmues1}")
print(f"La error estándar de la muestra 1 es: {errormues1}")
print(f"El intervalo de confianza de la muestra 1 es: {interconfmues1}")

El tamaño de la muestra 1 es: 197
La proporción de la muestra 1 es: 0.2131979695431472
La error estándar de la muestra 1 es: 0.026158513962325436
El intervalo de confianza de la muestra 1 es: (0.15049119159993335, 0.27590474748636107)


In [503]:
print(f"El tamaño de la muestra 2 es: {tammues}")
print(f"La proporción de la muestra 2 es: {propmues2}")
print(f"La error estándar de la muestra 2 es: {errormues2}")
print(f"El intervalo de confianza de la muestra 2 es: {interconfmues2}")

El tamaño de la muestra 2 es: 278
La proporción de la muestra 2 es: 0.19424460431654678
La error estándar de la muestra 2 es: 0.025267652387311962
El intervalo de confianza de la muestra 2 es: (0.13153782637333294, 0.2569513822597606)


$\textbf{Conclusiones:}$ El tamaño de la muestra crece cuando el margen de error es muy pequeño y la proporcion varia mucho por la probabilidad de error.